<a href="https://colab.research.google.com/github/ze978/PyTorch/blob/main/cnn_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
# 이미지데이터를 사용하기 위해서 기존 데이터 사용, 변형
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
learning_rate=0.001
epochs=30
batch_size=256


In [ ]:
mnist_train=dataset.MNIST(root="data/MNIST_data", #데이터 저장위치
                          train=True, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )
mnist_test=dataset.MNIST(root="data/MNIST_data", #데이터 저장위치
                          train=False, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )

100%|██████████| 9912422/9912422 [00:00<00:00, 39049152.95it/s]


Extracting data/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST_data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 67673571.97it/s]

Extracting data/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST_data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 44177449.71it/s]


Extracting data/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 20159289.70it/s]


Extracting data/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST_data/MNIST/raw



In [ ]:
data_loader=DataLoader(dataset=mnist_train,
                       batch_size=batch_size,
                       shuffle=True,
                       drop_last=True #뽑은거는 중복사용안함
                       )
test_loader=DataLoader(dataset=mnist_test,batch_size=batch_size,shuffle=True,drop_last=True)


In [ ]:
cnt_batch_train=len(data_loader) #234
cnt_batch_test=len(test_loader) #39
print(cnt_batch_train,cnt_batch_test)


234 39


In [ ]:
# 한번에 가져오기
for X, Y in data_loader:
  print(X.size(),Y.size())
  break

torch.Size([256, 1, 28, 28]) torch.Size([256])


In [ ]:
# 모델 클래스 정의
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    # 첫번째 레이어 합성곱연산
    self.layer1=nn.Sequential(
    nn.Conv2d(1,32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
    # 위의 작업 이후에 [32,14,14] 변환됨
    self.layer2=nn.Sequential(
      nn.Conv2d(32,64,kernel_size=3, padding=1, stride=1),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    # 위의 작업이 끝나면 [64,7,7]로 변환됨
    self.fc=nn.Linear(7*7*64, 10, bias=True)
    nn.init.xavier_uniform_(self.fc.weight) #완전연결층 한정 가중치 초기화

  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=out.view(out.size(0),-1)
    out=self.fc(out)
    return out


In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
model=CNN().to(device)
crit=nn.CrossEntropyLoss().to(device)
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  avg_cost=0

  for X,Y in data_loader:
    X=X.to(device)
    Y=Y.to(device)

    y_hat=model(X)
    optimizer.zero_grad()
    cost=crit(y_hat,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/cnt_batch_train
  print('epoch:{}, cost:{}'.format(epoch,avg_cost.item()))


epoch:0, cost:0.363978773355484
epoch:1, cost:0.08695533126592636
epoch:2, cost:0.05918367952108383
epoch:3, cost:0.04926785081624985
epoch:4, cost:0.041800517588853836
epoch:5, cost:0.036799851804971695
epoch:6, cost:0.03068341128528118
epoch:7, cost:0.02789015881717205
epoch:8, cost:0.024363527074456215
epoch:9, cost:0.023016195744276047
epoch:10, cost:0.020491762086749077
epoch:11, cost:0.018048515543341637
epoch:12, cost:0.015751441940665245
epoch:13, cost:0.013427306897938251
epoch:14, cost:0.012332289479672909
epoch:15, cost:0.012063338421285152
epoch:16, cost:0.009164007380604744
epoch:17, cost:0.007907208986580372
epoch:18, cost:0.008336317725479603
epoch:19, cost:0.007624950725585222
epoch:20, cost:0.006166087929159403
epoch:21, cost:0.006871819030493498
epoch:22, cost:0.006181670352816582
epoch:23, cost:0.004689824767410755
epoch:24, cost:0.004494091495871544
epoch:25, cost:0.0059734731912612915
epoch:26, cost:0.0031408476643264294
epoch:27, cost:0.0023104087449610233
epoch:2

In [ ]:
with torch.no_grad():
  accuracys=0

  for X, Y in test_loader:
    X=X.to(device)
    Y=Y.to(device)

    pred=model(X)
    accuracy=(pred.argmax(dim=1)==Y).sum()
    accuracys+=accuracy
    print(pred.argmax(dim=1))
    print(Y)
    print(accuracy)
  print(accuracy/cnt_batch_test*batch_size)


tensor([5, 3, 6, 9, 3, 8, 4, 2, 7, 4, 8, 6, 7, 7, 6, 8, 7, 5, 9, 5, 9, 4, 7, 4,
        2, 8, 6, 7, 3, 9, 5, 0, 2, 2, 9, 0, 6, 9, 1, 7, 4, 1, 9, 6, 9, 6, 4, 6,
        5, 0, 7, 6, 4, 7, 9, 3, 7, 3, 5, 1, 6, 9, 8, 7, 0, 8, 5, 7, 2, 9, 5, 1,
        7, 8, 7, 2, 5, 4, 9, 5, 5, 5, 6, 2, 4, 6, 9, 2, 3, 7, 7, 2, 2, 9, 8, 9,
        6, 2, 1, 1, 9, 4, 1, 7, 4, 1, 1, 0, 2, 3, 0, 2, 7, 5, 1, 0, 9, 0, 2, 2,
        5, 0, 8, 6, 1, 4, 2, 2, 7, 4, 2, 5, 0, 1, 4, 0, 6, 9, 2, 2, 4, 8, 7, 9,
        7, 8, 2, 0, 7, 8, 7, 7, 4, 8, 1, 6, 5, 7, 7, 6, 5, 6, 3, 4, 8, 0, 6, 4,
        0, 1, 1, 5, 0, 9, 6, 3, 6, 1, 0, 9, 7, 1, 6, 0, 4, 7, 9, 0, 0, 3, 4, 7,
        8, 1, 0, 9, 5, 3, 7, 5, 2, 7, 0, 6, 7, 0, 7, 4, 2, 0, 2, 1, 3, 9, 5, 1,
        3, 9, 8, 6, 6, 2, 5, 6, 9, 7, 2, 9, 8, 7, 3, 6, 6, 9, 8, 5, 0, 0, 9, 7,
        8, 4, 9, 5, 1, 1, 1, 9, 6, 7, 9, 1, 7, 2, 6, 9], device='cuda:0')
tensor([5, 3, 6, 9, 3, 8, 4, 2, 7, 4, 8, 6, 7, 7, 6, 8, 7, 5, 9, 5, 9, 4, 7, 4,
        2, 8, 6, 7, 3, 9, 5, 0, 2, 2, 9, 0, 6,

In [ ]:
# import pandas as pd
# from sklearn.metrics import confusion_matrix
# pd.DataFrame(confusion_matrix(Y[-1], torch.argmax(y_hat, dim=-1)),
#              index=['true_%d' % i for i in range(10)],
#              columns=['pred_%d' % i for i in range(10)])

In [ ]:
fashion_train=dataset.FashionMNIST(root="data/FashionMNIST_data", #데이터 저장위치
                          train=True, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )
fashion_test=dataset.FashionMNIST(root="data/FashionMNIST_data", #데이터 저장위치
                          train=False, #train data로 지정
                          transform=transforms.ToTensor(), #array -> toTensor로 변환
                          download=True
                          )